In [13]:
from pyspark.sql import SparkSession
import json
from confluent_kafka import Consumer, KafkaError

In [14]:
spark = SparkSession.builder.appName("MQTT-Kafka").getOrCreate()

In [15]:
spark

In [16]:
# Kafka consumer configuration
consumer_config = {
    'bootstrap.servers': 'kafka:9092',
    'group.id': 'jupyter-consumer-group',
    'auto.offset.reset': 'earliest'
}

# Create Kafka consumer
consumer = Consumer(consumer_config)

# Subscribe to the topic
consumer.subscribe(['AIDHOOSTATION'])

In [17]:
# Create a streaming DataFrame from Kafka
df = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", consumer_config['bootstrap.servers']) \
    .option("subscribe", "AIDHOOSTATION") \
    .option("group.id", consumer_config['group.id']) \
    .option("auto.offset.reset", consumer_config['auto.offset.reset']) \
    .load()

In [7]:
# Select and display the raw data (key and value are in binary format)
df_raw = df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)", "topic", "partition")

In [11]:
df_raw.columns

['key', 'value', 'topic', 'partition']

In [9]:
# Display the streaming DataFrame
query = df_raw \
    .writeStream \
    .outputMode("append") \
    .format("console") \
    .start()

# Keep the streaming job running
query.awaitTermination()

25/03/13 09:22:48 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-337f33c0-adce-46a7-a032-20ef11fb3c16. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/03/13 09:22:48 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/03/13 09:22:49 ERROR MicroBatchExecution: Query [id = 7cdfc66b-49cf-4653-9243-d90f2a8ed9cf, runId = e703fa8a-d88d-4e95-8b82-98a83ec114d6] terminated with error
java.lang.NoClassDefFoundError: org/apache/spark/kafka010/KafkaConfigUpdater
	at org.apache.spark.sql.kafka010.KafkaSourceProvider$.kafkaParamsForDriver(KafkaSourceProvider.scala:643)
	at org.apache.spark.sql.kafka010.KafkaSourceProvider$KafkaScan.toMicroBatchStream(KafkaSourceProvider.scala:482)
	at org.ap

StreamingQueryException: org/apache/spark/kafka010/KafkaConfigUpdater
=== Streaming Query ===
Identifier: [id = 7cdfc66b-49cf-4653-9243-d90f2a8ed9cf, runId = e703fa8a-d88d-4e95-8b82-98a83ec114d6]
Current Committed Offsets: {}
Current Available Offsets: {}

Current State: INITIALIZING
Thread State: RUNNABLE